In [1]:
import matplotlib
matplotlib.use('SVG')
# cd to base directory of this repository to enable the loading of NEURON simulations
import os
os.chdir('/home/david/Projects/osmorph/')
import numpy as np
import pandas as pd
from neuron import h
from neurostim.cell import Cell
from neurostim.models import L5_catV1
from neurostim.stimulator import Stimulator
from neurostim.simulation import SimControl
from neurostim.utils import convert_polar_to_cartesian_xz, interpolate
import matplotlib.pyplot as plt

In [2]:
# model config 1 ---- *** DO NOT RERUN ***
# NEURON simulator setup is corrupted if reinitialized in a running kernel, rerun complete notebook

# NEURON setup
h.load_file("stdrun.hoc")
h.cvode_active(1)

# load cell and create stimulation object
cell = Cell(
    model = eval('L5_catV1()'),
    ChR_soma_density=13e9,
    ChR_distribution='uniform'
)

# init stimulator
stimulator = Stimulator(
    diameter_um=200,
    NA=0.22,
)
# init simulation
simcontrol = SimControl(cell=cell, stimulator=stimulator)

	1 
	1 
	1 
	1 


In [3]:
# need to simulate at different intensities:
def simulate(intensity):
    sim_data = simcontrol.run(
        temp_protocol=dict(
            duration_ms=250,
            delay_ms=290,
            total_rec_time_ms=800
        ),
        stim_location=(0,0,0),
        stim_intensity_mWPERmm2=intensity,
        rec_vars = [['time [ms]', 'V_soma(0.5) [mV]'],
                    ['h._ref_t', 'h.soma(0.5)._ref_v']],
        interpol_dt_ms=0.1
    )
    return sim_data

# need to create trace plot
def plot(data):
    intensities = data.reset_index().intensity.unique()
    l_ = len(intensities)
    fig, axs = plt.subplots(nrows=len(intensities), figsize=(4,2))
    for i, intensity in enumerate(intensities):
        measurement = data.loc[intensity]
        axs[l_-1-i].plot(measurement['time [ms]'],measurement['V_soma(0.5) [mV]'], 
                      color='black', lw=0.5)
        axs[l_-1-i].set_ylim(-90,50)
        axs[l_-1-i].set_xlim(0,800)
        # Hide the right and top spines
        for pos in ['bottom', 'left', 'right', 'top']:
            axs[l_-1-i].spines[pos].set_visible(False)
    for ax in axs:
        ax.set_xticks([])
        ax.set_yticks([])
    return fig, axs    

In [4]:
# Photon intensities in experimental data are:
photon_intensities_cell1 = [4e14, 2e16, 1e17, 3e17] # panel A top
norm_int_cell1 = [i/photon_intensities_cell1[0] for i in photon_intensities_cell1]
print("Experimental data photon intensity steps:")
print("Cell 1:")
print(norm_int_cell1)

Experimental data photon intensity steps:
Cell 1:
[1.0, 50.0, 250.0, 750.0]


In [5]:
# scale factor needed to account for different light losses in the experimental setup vs virtual setup underlying the compuatational model
# (found through testing to yield similar stimulation characterisitcs)
intensity_scale = 1.8
intensities1 = np.array(norm_int_cell1) * intensity_scale
data = []
for intensity in intensities1:
    tmp = simulate(intensity)
    tmp['intensity'] = intensity
    data.append(tmp)
plot(pd.concat(data).set_index(['intensity']))
plt.savefig('figures/fig2/simulated_traces_1.svg')

In [6]:
def convert_input_intensity_to_soma_photon_intensity(intensity):
    """ input intensity at source in mW/mm2/s
        output intensity at soma in photons/cm2/s
    """
    light_power = intensity * 0.1**2 * np.pi *1e-3 # light power in W
    light_source = Stimulator(
        diameter_um=200,
        NA=0.22,
    )
    input_power_to_soma_intensity_factor = light_source.calculate_Tx_at_pos(
        [0,0,0],[0,0,h.soma(0.5).z_chanrhod])
    # photon energy:
    h_planck = 6.62607015e-34 #Js
    lamda = 450e-9
    c = 3e8
    E_photon = h_planck * c/lamda
    return light_power * input_power_to_soma_intensity_factor / E_photon
    

print('intensities1 [cm-2s-1]','------------')
for i in intensities1:
    print('{:.1E}'.format(convert_input_intensity_to_soma_photon_intensity(i)))

intensities1 [cm-2s-1] ------------
5.9E+15
2.9E+17
1.5E+18
4.4E+18


In [1]:
5.9e15/4e14

14.75